In [ ]:
"""

Compare the melt resulting from old and new tuning for last 10 years of
OPM026,OPM031,ctrl94,isf94

"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import glob
import seaborn as sns

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
home_path = '/bettik/burgardc/'

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)
    
plot_path='/bettik/burgardc/PLOTS/summer_paper_plots/'

READ IN AND FORMAT RIGNOT TABLE

In [ ]:
inputpath_rignot = home_path+'/DATA/SUMMER_PAPER/raw/'
file_rignot = pd.read_csv(inputpath_rignot + 'Rignot_data_withoutfootnotes.csv',delimiter=';',header=[0],index_col=0).dropna(how='all',axis=1)

In [ ]:
rignot_list_val = []
rignot_list_unc = []
nisf_list = []
for kisf in file_isf.Nisf:
    nname = file_isf['isf_name'].sel(Nisf=kisf).values
    
    if nname == 'Vincennes Bay':
        rignot_row = file_rignot.loc['Vincennes [8]']
    elif nname == 'Roi Baudouin':
        rignot_row = file_rignot.loc['Baudouin [9]']
    elif nname == 'Atka':
        rignot_row = file_rignot.loc['Atka [8]']
    elif nname == 'Moscow Univ.':
        rignot_row = file_rignot.loc['Moscow University']
    elif nname == 'Tracy Tremenchus':
        rignot_row = file_rignot.loc['Tracy/Tremenchus']
    elif nname == 'Stancomb Brunt':
        rignot_row = file_rignot.loc['Brunt/Stancomb']
    elif nname == 'Filchner-Ronne':
        rignot_row = file_rignot.loc['Ronne']
    elif nname == 'Ross':
        rignot_row = file_rignot.loc['Ross East']
    else:
        rignot_row = file_rignot.loc[nname]
    GL_flux = xr.DataArray(data=np.array(rignot_row['Grounding line flux'].split('±')[0]))
    GL_flux_unc = xr.DataArray(data=np.array(rignot_row['Grounding line flux'].split('±')[1]))
    Calv_flux = xr.DataArray(data=np.array(rignot_row['Ice-front flux'].split('±')[0]))
    Calv_flux_unc = xr.DataArray(data=np.array(rignot_row['Ice-front flux'].split('±')[1]))
    SMB = xr.DataArray(data=np.array(rignot_row['SMB'].split('±')[0]))
    SMB_unc = xr.DataArray(data=np.array(rignot_row['SMB'].split('±')[1]))
    SMB = xr.DataArray(data=np.array(rignot_row['SMB'].split('±')[0]))
    SMB_unc = xr.DataArray(data=np.array(rignot_row['SMB'].split('±')[1]))
    BMB = xr.DataArray(data=np.array(rignot_row['B'].split('±')[0]))
    BMB_unc = xr.DataArray(data=np.array(rignot_row['B'].split('±')[1]))
    val_ds = xr.merge([GL_flux.rename('GL_flux'),Calv_flux.rename('Calv_flux'),SMB.rename('SMB'),BMB.rename('Basal_melt')]).assign_coords({'Nisf': kisf})
    unc_ds = xr.merge([GL_flux_unc.rename('GL_flux'),Calv_flux_unc.rename('Calv_flux'),SMB_unc.rename('SMB'),BMB_unc.rename('Basal_melt')]).assign_coords({'Nisf': kisf})
    rignot_list_val.append(val_ds.astype(float))
    rignot_list_unc.append(unc_ds.astype(float))

rignot_val_all = xr.concat(rignot_list_val, dim='Nisf')
rignot_unc_all = xr.concat(rignot_list_unc, dim='Nisf')


READ IN ABUMIP FLUX

In [ ]:
inputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'

GL_fluxes = xr.open_dataset(inputpath_GL + 'all_GL_fluxes.nc')

PLOT

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [ ]:
GL_fluxes

In [ ]:
plt.figure(figsize=(8.25, 8.25/2))

for kisf in sorted_isf_rignot:
    rignot_val_kisf = rignot_val_all.sel(Nisf=kisf)
    flux_sum = - rignot_val_kisf['SMB'] + rignot_val_kisf['Calv_flux'] + rignot_val_kisf['Basal_melt']
    ABUMIP_pres_kisf = GL_fluxes['flux_Gt_ABUMIP'].sel(Nisf=kisf)
    
    plt.scatter(kisf, flux_sum, c='deepskyblue')
    plt.scatter(kisf, ABUMIP_pres_kisf, c='orange')
plt.xticks(ticks=sorted_isf_rignot, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values,rotation=90)
plt.tight_layout()
sns.despine()

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

f = plt.figure()
f.set_size_inches(8.25*2, 8.25*2)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    rignot_val_kisf = rignot_val_all.sel(Nisf=kisf)
    flux_sum = - rignot_val_kisf['SMB'] + rignot_val_kisf['Calv_flux'] + rignot_val_kisf['Basal_melt']
    
    ax[i].axhline(y=GL_fluxes['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='orange')
    ax[i].axhline(y=rignot_val_kisf['GL_flux'], color='k')
    ax[i].axhline(y=GL_fluxes['flux_Gt_ref'].sel(Nisf=kisf), color='grey',linestyle='--')
    ax[i].axhline(y=flux_sum, color='deepskyblue')


    



    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'comparison_rignot_ABUMIPref.pdf')